# Report on the number of slices used per mouse

In [1]:
#import numpy then pandas

import numpy as np
import seaborn as sns
from datetime import datetime

import pg8000          #pg8000 access SQL databases
import pandas as pd    #pandas will be needed to work in a dataframe

### LIMS Access code by Agata

In [2]:
#Opens LIMS, makes a query and then closes LIMS after

def _connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    conn = pg8000.connect(user=user, host=host, database=database, password=password, port=port)
    return conn, conn.cursor()

def _select(cursor, query):
    cursor.execute(query)
    columns = [ d[0] for d in cursor.description ]
    return [ dict(zip(columns, c)) for c in cursor.fetchall() ]

def limsquery(query, user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    """A function that takes a string containing a SQL query, connects to the LIMS database and outputs the result."""
    conn, cursor = _connect(user, host, database, password, port)
    try:
        results = _select(cursor, query)
    finally:
        #THESE ARE IMPORTANT!!!!!! (Every query needs to be closed when done)
        #This closes query
        cursor.close()             
        conn.close()
    return results


#This last function will take our query results and put them in a dataframe so that they are easy to work with
def get_lims_dataframe(query):
    '''Return a dataframe with lims query'''
    result = limsquery(query)
    try:
        data_df = pd.DataFrame(data=result, columns=result[0].keys())
    except IndexError:
        print "Could not find results for your query."
        data_df = pd.DataFrame()
    return data_df

##### This is how to get the column names of each table

In [5]:
my_query = "SELECT * FROM specimens LIMIT 10"
#we are going to select all columns in the specimens table and limit our search to the first 10
my_result = limsquery(my_query)
first_element = my_result[0]
#now we only want to look at the first element of our result
print first_element.keys()

['cell_depth', 'ephys_roi_result_id', 'parent_y_coord', 'reference_space_id', 'updated_at', 'cell_label', 'preparation_method_id', 'parent_x_coord', 'location_id', 'id', 'cortex_layer_id', 'plane_of_section_id', 'frozen_at', 'flipped_specimen_id', 'data', 'pinned_radius', 'rna_integrity_number', 'histology_well_name', 'created_by', 'priority', 'parent_id', 'ephys_start_time_sec', 'project_id', 'alignment3d_id', 'carousel_well_name', 'patched_cell_container', 'updated_by', 'cell_prep_id', 'biophysical_model_state', 'barcode', 'storage_directory', 'x_coord', 'tissue_ph', 'specimen_preparation_method_id', 'donor_id', 'starter_cell_count', 'operation_id', 'ephys_neural_tissue_plan_id', 'structure_id', 'parent_z_coord', 'facs_well_id', 'name', 'y_coord', 'normalization_group_id', 'postmortem_interval_id', 'specimen_set_id', 'created_at', 'tissue_processing_id', 'ephys_cell_plan_id', 'hemisphere_id', 'ephys_qc_result', 'cell_reporter_id', 'task_flow_id', 'external_specimen_name']


In [7]:
my_query = 'SELECT name, histology_well_name, patched_cell_container FROM specimens'
lims_df = get_lims_dataframe(my_query)
lims_df.tail(50)

,patched_cell_container,histology_well_name,name
458048,None,EPhys_180226_03_C05_M,Ndnf-IRES2-dgCre;Ai14-377316.06.02
458049,None,Ephys_180226_03_C06_M,Ndnf-IRES2-dgCre;Ai14-377316.05.01
458050,None,EPhys_180226_03_D01_M,Vip-IRES-Cre;Ai14-377300.03.01
458051,None,EPhys_180226_03_D05_M,Vip-IRES-Cre;Ai14-377300.05.02
458052,None,Ephys_180226_03_D06_M,Slc17a6-IRES-Cre;Ai14-376996.04.01
458053,None,EPhys_180226_04_A03_M,Sst-IRES-Cre;Ai14-376206.04.01
458054,None,EPhys_180226_04_A05_M,Sst-IRES-Cre;Ai14-376206.04.02
458055,None,EPhys_180226_04_A06_M,Sst-IRES-Cre;Ai14-376206.06.02
458056,None,EPhys_180226_04_B01_M,Sst-IRES-Cre;Ai14-376206.05.01
458057,None,multi_180226_21_A01,Ntsr1-Cre_GN220;Ai140;Sst-IRES-FlpO-377968-Ai6...


In [ ]:
#Necessity:
#date of slice 
#specimen name 
#MAIN -- histology_well_name
#Maybe for specifying user -- patched_cell_container

In [ ]:
#Usually 12 slices in each carousel 
#6 Left and 6 Right hemispheres
#Left Hemi = .01
#Right Hemi = .02
#Slice numbers = 1-6 

#### Creating functions

In [ ]:
def slice_per_mouse():
    